In [12]:
import os
import pandas as pd
import numpy as np

from dask import dataframe as dd
from ray import data
import modin.pandas as mpd

In [8]:
# using pandas

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [9]:
print('-' * 80)
print('train')
train = import_data('D:\\FSDS-iNeuron\\3.Resource\\Dataset\\Placement_Task\\Click_ThroughRatePrediction\\train.csv')

--------------------------------------------------------------------------------
train


MemoryError: Unable to allocate 1.20 GiB for an array with shape (4, 40428967) and data type int64

In [28]:
# Using chunksize => pandas

df = pd.read_csv('D:\\FSDS-iNeuron\\3.Resource\\Dataset\\Placement_Task\\Click_ThroughRatePrediction\\train.csv', chunksize=100)

df

In [29]:

chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in pd.read_csv('D:\\FSDS-iNeuron\\3.Resource\\Dataset\\Placement_Task\\Click_ThroughRatePrediction\\train.csv', chunksize=100):  
    # perform data filtering 
    chunk_filter = chunk_preprocessing(chunk)
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

NameError: name 'chunk_preprocessing' is not defined

2023-05-28 13:18:17,842	WARNING worker.py:1986 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffffea6c96123630721a8d663c1001000000 Worker ID: 8f11dd2ca648970d3635a64cad8ed2ed64c58139b07456983146fd3d Node ID: a61ed8e6137ebef287362cb29449c2a599c037cae9dfa60f883804a7 Worker IP address: 127.0.0.1 Worker port: 57346 Worker PID: 2236 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 10054. An existing connection was forcibly closed by the remote host. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


In [4]:
# Using dask

dd.read_csv('D:\\FSDS-iNeuron\\3.Resource\\Dataset\\Placement_Task\\Click_ThroughRatePrediction\\train.csv')

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
npartitions=98,,,,,,,,,,,,,,,,,,,,,,,,
,uint64,int64,int64,int64,int64,object,object,object,object,object,object,object,object,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
# Using Ray

data.read_csv('D:\\FSDS-iNeuron\\3.Resource\\Dataset\\Placement_Task\\Click_ThroughRatePrediction\\train.csv', parallelism=100)

2023-05-28 12:01:36,755	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2023-05-28 12:01:46,213	WARNING read_api.py:347 -- ⚠️  The blocks of this dataset are estimated to be 11.0x larger than the target block size of 512 MiB. This may lead to out-of-memory errors during processing. Consider reducing the size of input files or using `.repartition(n)` to increase the number of dataset blocks.
2023-05-28 12:01:46,244	WARNING formatters.py:922 -- Missing packages: ['ipywidgets']. Run `pip install ipywidgets` for rich notebook output.


In [20]:
# Using Modin

data = 'D:\\FSDS-iNeuron\\3.Resource\\Dataset\\Placement_Task\\Click_ThroughRatePrediction\\train.csv'
df = mpd.DataFrame(data)

ValueError: DataFrame constructor not properly called!

In [23]:
pip install vaex

     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     -------------------- ----------------- 30.7/58.3 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 58.3/58.3 kB 763.6 kB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached httptools-0.5.0-cp38-cp38-win_amd64.whl (144 kB)
     ---------------------------------------- 0.0/124.7 kB ? eta -:--:--
     -------------------------------------- 124.7/124.7 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/270.9 kB ? eta -:--:--
     -------------------- ----------------- 143.4/270.9 kB 2.8 MB/s eta 0:00:01
     -------------------------------------- 270.9/270.9 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [26]:
# Using vaex

import vaex

df = vaex.from_csv('D:\\FSDS-iNeuron\\3.Resource\\Dataset\\Placement_Task\\Click_ThroughRatePrediction\\train.csv')

MemoryError: Unable to allocate 256. KiB for an array with shape (32768,) and data type int64